# Project Title
### Data Engineering Capstone Project

#### Project Summary
--describe your project at a high level--

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [4]:
# Do all imports and installs here
import pandas as pd
import datetime
import numpy as np
from pandas.api.types import is_numeric_dtype

### Step 1: Scope the Project and Gather Data

#### Scope 
Explain what you plan to do in the project in more detail. What data do you use? What is your end solution look like? What tools did you use? etc>

#### Describe and Gather Data 
Describe the data sets you're using. Where did it come from? What type of information is included? 

In [5]:
# Read in the data here and print columns and types
# immigration_data small
df_immigration = pd.read_csv('data/immigration_data_sample.csv')
print( df_immigration.dtypes)

# temperature data
df_temperature = pd.read_csv('data/Global_temperature/GlobalLandTemperaturesByCity.csv')
print(df_temperature.dtypes)

# demographics data
df_dem = pd.read_csv('data/Us_city_demographics/us-cities-demographics.csv', delimiter = ';')
print(df_dem.dtypes)

# airport data
df_air = pd.read_csv('data/airport-codes_csv.csv')
print(df_air.dtypes)

# immigration data big file
df_imm_par = pd.read_parquet("data/sas_data")


Unnamed: 0      int64
cicid         float64
i94yr         float64
i94mon        float64
i94cit        float64
i94res        float64
i94port        object
arrdate       float64
i94mode       float64
i94addr        object
depdate       float64
i94bir        float64
i94visa       float64
count         float64
dtadfile        int64
visapost       object
occup          object
entdepa        object
entdepd        object
entdepu       float64
matflag        object
biryear       float64
dtaddto        object
gender         object
insnum        float64
airline        object
admnum        float64
fltno          object
visatype       object
dtype: object
dt                                object
AverageTemperature               float64
AverageTemperatureUncertainty    float64
City                              object
Country                           object
Latitude                          object
Longitude                         object
dtype: object
City                       object
State        

### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

#### Cleaning Steps
Document steps necessary to clean the data

In [6]:
start = datetime.datetime(1960, 1, 1)

# Performing cleaning tasks here
# demographics 
# remove race table aggregation over city will be conducted with in redshift
df_dem_clean = df_dem.drop(['Race','Count'], axis=1).drop_duplicates()
df_dem_clean.sort_values('City')

# rename columns with spaces
df_dem_clean = df_dem_clean.rename(columns={"Median Age":"Median_Age", "Male_Population":"Male_Population",
                       "Female Population":"Female_Population",
                       "Total Population": "Total_Population",
                       "Number of Veterans": "Number_of_Veterans",
                       "Average Household Size":"Average_Household_Size",
                       "State Code":"State_Code",
                       "Foreign-born": "Foreign_born"})

# back to csv write
df_dem_clean.to_csv('Data_clean/demographics_data.csv', index=False)




In [7]:
# immigration
# remove columns with high nan percentage
print(df_imm_par.isnull().sum() * 100 / len(df_imm_par))
df_immigration_clean = df_imm_par.drop(['visapost', 'occup', 'entdepu', 'insnum'], axis = 1)

# make sense of arrdate en depdate
df_immigration_clean['arrdate'] = pd.to_timedelta(df_immigration_clean['arrdate'], unit = 'day') + start
df_immigration_clean['arrdate'] = pd.to_datetime(df_immigration_clean['arrdate'], unit='day', origin = 'unix')
df_immigration_clean['depdate'] = pd.to_timedelta(df_immigration_clean['depdate'], unit = 'day') + start


# write back to parquet
df_immigration_clean.to_parquet('Data_clean/i94_unpar/unpar.snappy')


cicid        0.000000
i94yr        0.000000
i94mon       0.000000
i94cit       0.000000
i94res       0.000000
i94port      0.000000
arrdate      0.000000
i94mode      0.004241
i94addr      3.357000
depdate      3.862618
i94bir       0.000808
i94visa      0.000000
count        0.000000
dtadfile     0.000050
visapost    60.339714
occup       99.723006
entdepa      0.004191
entdepd      3.862315
entdepu     99.986620
matflag      3.862315
biryear      0.000808
dtaddto      0.006463
gender      14.553598
insnum      99.965666
airline      0.037060
admnum       0.000000
fltno        0.037666
visatype     0.000000
dtype: float64


In [6]:
# temperature data
def mean_str(col):
    if is_numeric_dtype(col):
        return col.mean()
    else:
        return col.unique() if col.nunique() == 1 else np.NaN
    
df_temp_clean = df_temperature

# change dt to datetime
df_temp_clean['dt']=pd.to_datetime(df_temp_clean['dt'])
df_temp_clean['dt']=pd.DatetimeIndex(df_temp_clean['dt']).month

# group by city and month
df_temp_clean= df_temp_clean.groupby(['dt', 'City']).agg(mean_str)

# write back to csv
df_temp_clean.to_csv('Data_clean/temperature_data.csv')
df_temperature

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
0,11,6.068,1.737,Århus,Denmark,57.05N,10.33E
1,12,NaN,NaN,Århus,Denmark,57.05N,10.33E
2,1,NaN,NaN,Århus,Denmark,57.05N,10.33E
3,2,NaN,NaN,Århus,Denmark,57.05N,10.33E
4,3,NaN,NaN,Århus,Denmark,57.05N,10.33E
...,...,...,...,...,...,...,...
8599207,5,11.464,0.236,Zwolle,Netherlands,52.24N,5.26E
8599208,6,15.043,0.261,Zwolle,Netherlands,52.24N,5.26E
8599209,7,18.775,0.193,Zwolle,Netherlands,52.24N,5.26E
8599210,8,18.025,0.298,Zwolle,Netherlands,52.24N,5.26E


In [9]:
# airport data
# where iata code exists
df_air_clean = df_air[df_air['iata_code'].notna()].drop(['gps_code', 'local_code'], axis = 1)

# and country is us
df_air_clean = df_air_clean[df_air_clean['iso_country']=='US']

# remove columns not needed
df_air_clean = df_air_clean.drop(['continent','iso_country', 'ident'], axis = 1)

# for region only keep the state code
df_air_clean["iso_region"] = df_air_clean["iso_region"].str.replace("US-", "")

# write back to csv
df_air_clean.to_csv('Data_clean/airport_data.csv', index=False)

### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [ ]:
# Write code here
# see airflow dags

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [ ]:
# Perform quality checks here

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.